In [ ]:
!pip install selenium

In [ ]:
!pip install webdriver-manager

In [ ]:
import os
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

# Disable SSL verification warnings
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--ignore-certificate-errors")  # Bypass SSL errors
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

In [ ]:
def get_kenyan_institutions():
    kenet_url = "https://www.kenet.or.ke/content/list-kenet-members"
    try:
        response = requests.get(kenet_url, verify=False)  # Bypass SSL verification
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        institutions = []
        for link in soup.find_all("a", href=True):
            if "ac.ke" in link["href"]:
                institutions.append((link.text.strip(), link["href"].strip()))
        return institutions
    except requests.exceptions.RequestException as e:
        print(f"Error fetching institutions from KENET: {e}")
        return []

institutions = get_kenyan_institutions()
print(f"Found {len(institutions)} institutions.")

In [16]:
def extract_elearning_info(url):
    try:
        response = requests.get(url, verify=False, timeout=10)  # Bypass SSL verification
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Extract potential e-learning links and platforms
        elearning_links = set()
        platforms = set()
        for link in soup.find_all("a", href=True):
            href = link["href"].lower()
            if any(keyword in href for keyword in ["moodle", "elearning", "lms", "virtual", "portal"]):
                elearning_links.add(href)
                if "moodle" in href:
                    platforms.add("Moodle")
                elif "elearning" in href:
                    platforms.add("E-learn")
                elif "lms" in href:
                    platforms.add("LMS")
                elif "virtual" in href:
                    platforms.add("Virtual Learning")
                elif "portal" in href:
                    platforms.add("E-learning Portal")
                else:
                    platforms.add("Unknown")
        
        # Extract emails and phone numbers
        emails = list(set(re.findall(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", response.text)))
        phones = list(set(re.findall(r"\+254\d{9}|07\d{8}|01\d{8}", response.text)))
        
        return list(elearning_links), list(platforms), emails, phones
    except requests.exceptions.RequestException as e:
        print(f"Error scraping {url}: {e}")
        return [], [], [], []

In [17]:
def main():
    institutions = get_kenyan_institutions()
    all_data = []
    
    for name, url in institutions:
        elearning_links, platforms, emails, phones = extract_elearning_info(url)
        all_data.append([name, url, elearning_links, platforms, emails, phones])
    
    df = pd.DataFrame(all_data, columns=["Institution Name", "Website URL", "E-learning URLs", "E-learning Platform", "Emails", "Phone Numbers"])
    df.to_csv("kenyan_institutions.csv", index=False)
    print("Scraping completed. Data saved to kenyan_institutions.csv")

if __name__ == "__main__":
    main()

Error scraping https://tuc.ac.ke/: 403 Client Error: Forbidden for url: https://tuc.ac.ke/
Error scraping https://bushiangalatechnical.ac.ke/: 403 Client Error: Forbidden for url: https://bushiangalatechnical.ac.ke/
Error scraping https://fck.ac.ke/: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error scraping https://kinyanjuitechnical.ac.ke/: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error scraping https://rvti.ac.ke/new/: 403 Client Error: Forbidden for url: https://rvti.ac.ke/new/
Error scraping https://wotetti.ac.ke/: 403 Client Error: Forbidden for url: https://wotetti.ac.ke/
Error scraping https://mmtvc.ac.ke/: 406 Client Error: Not Acceptable for url: https://mmtvc.ac.ke/
Error scraping https://www.ngongtvc.ac.ke/: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Error scraping https://kmtc.ac.ke/: HTTPSConnectionPool(host='kmtc.ac.ke', port=443): Read